In [1]:
import requests
from pprint import pprint
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

tmdb_key = "6971e619144092b46f0572e9faf949de"

In [4]:
start_year = 1927
end_year = 2020
year = 0

years = [year for year in range(start_year,end_year+1)]
    

In [5]:
movie_id = []
movie_title = []

base_url = f"https://api.themoviedb.org/3/discover/movie?api_key={tmdb_key}&primary_release_year="

sort_by = "&sort_by=revenue.desc"

revenue_url = f"{base_url}{year}{sort_by}"

for year in years:
    
    year_json = requests.get(f"{base_url}{year}{sort_by}").json()
    
    movie_id.append(year_json['results'][0]['id'])
    movie_title.append(year_json['results'][0]['original_title'])

In [6]:
df = pd.DataFrame ({
    'Year': years,
    'Movie ID': movie_id,
    'Movie Title': movie_title    
})

In [7]:
df.head()

,Year,Movie ID,Movie Title
0,1927,287262,The Battles of the Coronel and Falkland Islands
1,1928,173689,Lights of New York
2,1929,65203,The Broadway Melody
3,1930,166307,Tom Sawyer
4,1931,3035,Frankenstein


In [8]:
films_df = df.loc[:,['Year','Movie ID','Movie Title']]

films_df['Revenue'] = ""

films_df.head()

,Year,Movie ID,Movie Title,Revenue
0,1927,287262,The Battles of the Coronel and Falkland Islands,
1,1928,173689,Lights of New York,
2,1929,65203,The Broadway Melody,
3,1930,166307,Tom Sawyer,
4,1931,3035,Frankenstein,


In [16]:
#Then use the above df to pass the film id's through the below url and grab the gross revenue and add it to the df

base_url2 = f"https://api.themoviedb.org/3/movie/"
#Id goes here
api_key = f"?api_key="
#tmdb_key goes here
language = f"&language=en-US"

for index, row in films_df.iterrows():
    
    query_url = base_url2 + str(row['Movie ID']) + api_key + tmdb_key + language
    
    response_url = requests.get(query_url).json()
    
    try:
        films_df.loc[index, 'Revenue'] = response_url['revenue']
    
    except (KeyError):
        print(f"No Revenue For this film - '{row['Movie Title']}'")
    

No Revenue For this film - 'The Last Of Us Part II'


In [17]:
films_df

,Year,Movie ID,Movie Title,Revenue
0,1927,287262,The Battles of the Coronel and Falkland Islands,6241682
1,1928,173689,Lights of New York,1200000
2,1929,65203,The Broadway Melody,4358000
3,1930,166307,Tom Sawyer,11000000
4,1931,3035,Frankenstein,12000000
...,...,...,...,...
89,2016,271110,Captain America: Civil War,1153296293
90,2017,181808,Star Wars: The Last Jedi,1332539889
91,2018,299536,Avengers: Infinity War,2046239637
92,2019,299534,Avengers: Endgame,2797800564


In [52]:
pprint(film_details)

{'adult': False,
 'backdrop_path': '/5Iw7zQTHVRBOYpA0V6z0yypOPZh.jpg',
 'belongs_to_collection': {'backdrop_path': '/d8duYyyC9J5T825Hg7grmaabfxQ.jpg',
                           'id': 10,
                           'name': 'Star Wars Collection',
                           'poster_path': '/r8Ph5MYXL04Qzu4QBbq2KjqwtkQ.jpg'},
 'budget': 200000000,
 'genres': [{'id': 878, 'name': 'Science Fiction'},
            {'id': 28, 'name': 'Action'},
            {'id': 12, 'name': 'Adventure'}],
 'homepage': 'https://www.starwars.com/films/star-wars-episode-viii-the-last-jedi',
 'id': 181808,
 'imdb_id': 'tt2527336',
 'original_language': 'en',
 'original_title': 'Star Wars: The Last Jedi',
 'overview': 'Rey develops her newly discovered abilities with the guidance of '
             'Luke Skywalker, who is unsettled by the strength of her powers. '
             'Meanwhile, the Resistance prepares to do battle with the First '
             'Order.',
 'popularity': 42.653,
 'poster_path': '/kOVEVeg59